In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Unlock FHIR with RAG on Vertex AI

## {TO-DO Update Links} Run the Notebook

**_NOTE_**: This notebook has been tested in the following environment:

* Python version = 3.10.13

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/language/use-cases/document-qa/question_answering_documents_langchain_matching_engine.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Run in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/language/use-cases/document-qa/question_answering_documents_langchain_matching_engine.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/blob/main/language/use-cases/document-qa/question_answering_documents_langchain_matching_engine.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>
</table>

| | |
|-|-|
|Author(s) | [Vikrama Adethyaa](https://github.com/adethyaa) |

---
## Overview

This notebook demonstrates building a natural language interface to complex [FHIR](https://fhir.org/about.html) datasets using [Google Cloud’s Vertex AI](https://cloud.google.com/vertex-ai?hl=en).  Leveraging Retrieval Augmented Generation (RAG), Enterprise Knowledge Graphs, and Vector Search, this solution empowers healthcare professionals to query FHIR data with natural language. This demo is inspired by [Sam Schifman's work](https://medium.com/@samschifman/rag-on-fhir-with-knowledge-graphs-04d8e13ee96e).

---
## Environment Setup

### Set up your Google Cloud project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

2. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

3. [Enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

4. If you are running this notebook locally, you need to install the [Cloud SDK](https://cloud.google.com/sdk).

### Install Packages and Dependencies
Please install the following packages to run this notebook.

In [ ]:
# Install Vertex AI SDK
! pip install --user --upgrade google-cloud-aiplatform==1.43.0 

# Install Langchain 
! pip install --user --upgrade langchain==0.1.11 langchain-google-vertexai

# Install Neo4j
! pip install --user --upgrade neo4j==5.18.0

# For Matching Engine integration dependencies
! pip install --user --upgrade tensorflow tensorflow-hub tensorflow-text

### Download Matching Engine Helper Scripts

- ***Matching Engine*** is now called ***[Vertex AI Vector Search](https://cloud.google.com/vertex-ai/docs/vector-search/overview)***
- The cell below downloads helper functions necessary for the Vertex AI Matching Engine. These functions improve notebook readability. You can find the ***[source code on Github](https://github.com/GoogleCloudPlatform/generative-ai/tree/main/language/use-cases/document-qa/utils).***

In [ ]:
import os
import urllib.request

util_folder = "utils"

if not os.path.exists(util_folder):
    os.makedirs(util_folder)

url_prefix = "https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/language/use-cases/document-qa/utils"
files = ["__init__.py", "matching_engine.py", "matching_engine_utils.py"]

for fname in files:
    urllib.request.urlretrieve(f"{url_prefix}/{fname}", filename=f"{util_folder}/{fname}")

***Restart Kernel***

Run the following cell to restart the kernel or use the button to restart the kernel.

<div class="alert alert-block alert-warning">
<b>⚠️ Kindly allow the kernel to finish restarting before continuing. ⚠️</b>
</div>

In [1]:
# Automatically restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

### Authenticating your notebook environment

- If you are using **Colab** to run this notebook, run the cell below and continue.
- If you are using **Vertex AI Workbench**, check out the setup instructions [here](https://github.com/GoogleCloudPlatform/generative-ai/tree/main/setup-env)

In [1]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

**Vertex AI Workbench**
- Open a Terminal in the Jupyter notebook
- Execute the below command and follow the instructions

```bash
gcloud auth login
```

### Define Variables

In [2]:
# Limit FHIR files loaded for demo purposes
# Since Data Ingestion into Neo4J and Vector Search Index takes time, We set the parameter below to control the number of files ingested.
DEMO_FILES_INGEST_LIMIT = 20 # @param {type:"integer"}

# GCP Parameters
PROJECT_ID = "propane-crawler-363311"  # @param {type:"string"}
REGION = "us-central1"  # @param {type: "string"}

# Neo4J Connection Parameters
NEO4J_URL="bolt://localhost:7687" # @param {type:"string"}
NEO4J_USER="neo4j" # @param {type:"string"}
NEO4J_PASSWORD="password" # @param {type:"string"}

# Vertex AI Vector Search (MatchingEngine) Index Parameters
ME_INDEX_NAME = 'fhir_me_index'  # @param {type: "string"}
ME_EMBEDDING_GCS_DIR = f'{PROJECT_ID}-me-bucket' # @param {type:"string"} 
ME_DESCRIPTION = "Index for FHIR Resources" # @param {type:"string"} 

ME_ENHANCED_CONTEXT_INDEX_NAME = f'{ME_INDEX_NAME}_enhanced' # @param {type:"string"} 
ME_ENHANCED_EMBEDDING_GCS_DIR = f'{ME_EMBEDDING_GCS_DIR}_enhanced' # @param {type:"string"} 
ME_ENHANCED_DESCRIPTION = f'Enhanced Context {ME_DESCRIPTION}' # @param {type:"string"} 

# Dimension Vertex PaLM Text Embedding
ME_DIMENSIONS = 768 # @param {type:"integer"} 
ME_DISTANCE_MEASURE_TYPE = "DOT_PRODUCT_DISTANCE" # @param {type:"string"} 

# Update to bigger SHARDS for larger data volumes & performance
# Doc - https://cloud.google.com/vertex-ai/docs/vector-search/create-manage-index
ME_SHARD_SIZE = "SHARD_SIZE_SMALL" # @param ["SHARD_SIZE_SMALL", "SHARD_SIZE_MEDIUM", "SHARD_SIZE_LARGE"] 

# Vertex AI Vector Search (MatchingEngine) Endpoint Parameters
# Doc - https://cloud.google.com/vertex-ai/docs/vector-search/create-manage-index

# The machine types that you can use to deploy your index
ME_ENDPOINT_MACHINE_TYPE = "e2-standard-2" # @param ["n1-standard-16", "n1-standard-32", "e2-standard-2", "e2-standard-16", "e2-highmem-16", "n2d-standard-32"] 

ME_ENDPOINT_MIN_REPLICA_COUNT = 2 # @param {type:"integer"} 
ME_ENDPOINT_MAX_REPLICA_COUNT = 10 # @param {type:"integer"} 


# Set the LLM to use
VERTEX_AI_MODEL_NAME = 'gemini-1.0-pro-001'

### Import Libraries

- **Colab only:** Run the below cell to initialize the Vertex AI SDK. For Vertex AI Workbench, you don't need to run this.


In [3]:
import vertexai
vertexai.init(project=PROJECT_ID, location=REGION)

- Import Python Libraries

<div class="alert alert-block alert-warning">
<b>⚠️ Restart python kernel if issues while importing langchain  ⚠️</b>
</div>

In [4]:
# Utils
import os
import glob
from pprint import pprint
import json

import time
import datetime

import uuid
import numpy as np

import warnings

import logging
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)

from google.protobuf.json_format import MessageToDict

# Google Vertex AI
# from google.cloud import aiplatform
# print(f"Vertex AI SDK version: {aiplatform.__version__}")

# Neo4j Helper scripts
from utils.NEO4J_Graph import Graph
from utils.FHIR_to_string import FHIR_to_string
from utils.FHIR_to_graph import resource_to_node, resource_to_edges, flat_fhir_to_json_str

# Langchain
import langchain
print(f"LangChain version: {langchain.__version__}")

from langchain_google_vertexai import VertexAI
from langchain_google_vertexai import VertexAIEmbeddings
from langchain import PromptTemplate

# Import libraries
from langchain.chains import RetrievalQA
from langchain_google_vertexai import ChatVertexAI


# Import custom Matching Engine packages
from utils.matching_engine import MatchingEngine
from utils.matching_engine_utils import MatchingEngineUtils

# Import Custom LangChain Retriever
from utils.FHIRResourcesRetriever import FHIRResourcesRetriever

warnings.filterwarnings("ignore")

LangChain version: 0.1.11


---
## Fetch FHIR Synthetic Data

We will be using sample FHIR data from [Synthea](https://synthea.mitre.org/) for the purpose of this demonstration. 
We will be using the pre-generated data available [here](https://synthetichealth.github.io/synthea-sample-data/downloads/latest/synthea_sample_data_fhir_latest.zip). 

### Download synthetic FHIR Data from Synthea

In [ ]:
! mkdir -p $HOME/working

! curl  \
--url https://synthetichealth.github.io/synthea-sample-data/downloads/latest/synthea_sample_data_fhir_latest.zip \
--output $HOME/working/synthea_sample_data_fhir_latest.zip

- Unzip the synthea_sample_data_fhir_latest.zip 

In [ ]:
! unzip $HOME/working/synthea_sample_data_fhir_latest.zip -d $HOME/working/bundles

### Taking Inventory of FHIR Files

In [5]:
# Feth all FHIR Files
fhir_folder_path = './working/bundles'
fhir_files_list = glob.glob(f"{fhir_folder_path}/*.json")

# Limiting files to ingest
fhir_files_list.sort()
fhir_files_list = fhir_files_list[:DEMO_FILES_INGEST_LIMIT]


num_of_files = len(fhir_files_list)
print(f'Number of FHIR Files that would be ingested: {num_of_files}')

Number of FHIR Files that would be ingested: 20


In [6]:
file_counter = 1
total_resources = 0
file_resources_meta_list = []
resource_types_count = {}

for fhir_file in fhir_files_list:
    fhir_file_name = os.path.basename(fhir_file)
    # print(f'File {file_counter} of {num_of_files}: {fhir_file_name}')
    file_counter += 1
    
    with open(fhir_file) as raw:
        bundle = json.load(raw)
        resources_entry_list = bundle['entry']
            
        # Resources Counter
        num_of_resources = len(resources_entry_list)
        total_resources += num_of_resources
        file_resources_meta = {}
        file_resources_meta['file_name'] = fhir_file_name
        file_resources_meta['resources_count'] = num_of_resources
        
        file_resources_meta_list.append(file_resources_meta)
        
        # Count Individual Resource Types
        for entry in resources_entry_list:
            resource = entry['resource']
            resource_type = resource["resourceType"]

            if resource_type not in resource_types_count.keys():
                # print(f'Creating Dict entry for Resource: {resource_type}')
                resource_types_count[resource_type] = 0
            
            resource_types_count[resource_type] += 1

print(f'\nNumber of FHIR Files to process = {num_of_files}')            
print(f'Total Resources:{total_resources}\n')

print(f'Resource Types Count:')
pprint(resource_types_count)

print('\nFile Resources Meta:')
pprint(file_resources_meta_list)


Number of FHIR Files to process = 20
Total Resources:14060

Resource Types Count:
{'AllergyIntolerance': 23,
 'CarePlan': 66,
 'CareTeam': 66,
 'Claim': 1245,
 'Condition': 526,
 'Device': 23,
 'DiagnosticReport': 1451,
 'DocumentReference': 743,
 'Encounter': 743,
 'ExplanationOfBenefit': 1245,
 'ImagingStudy': 13,
 'Immunization': 299,
 'Medication': 63,
 'MedicationAdministration': 63,
 'MedicationRequest': 502,
 'Observation': 5252,
 'Patient': 20,
 'Procedure': 1513,
 'Provenance': 20,
 'SupplyDelivery': 184}

File Resources Meta:
[{'file_name': 'Adalberto916_Bernier607_a69d4bbb-0ddb-e8fc-0984-0ba9a27d961f.json',
  'resources_count': 718},
 {'file_name': 'Adela471_Escalante498_9d8f8194-a7a5-b8a5-8d7b-a14e5fa7ba3c.json',
  'resources_count': 545},
 {'file_name': 'Alejandra902_Murillo232_de08ab4a-7258-d0e3-e282-8d97e06e1dc7.json',
  'resources_count': 357},
 {'file_name': 'Alexis664_Schaefer657_d0d9bbbc-cd3e-f376-c55f-c6ed5f70fd3e.json',
  'resources_count': 269},
 {'file_name': 'A

---
## Load FHIR Data into Neo4J

### Deploy Neo4J Docker Container

We'll use **Docker** to deploy a local Neo4j instance for demo purposes.  For production environments, we highly recommend Neo4j on Google Cloud Marketplace. [Try Neo4j on Google Cloud Marketplace](https://console.cloud.google.com/marketplace/product/endpoints/prod.n4gcp.neo4j.io?pli=1&mpp=4bfb2414ab973c741b6f067bf06d5575&mpid=%24device%3A18e0c346ea25f9-098a968de268b5-1d525637-384000-18e0c346ea25fa).

The command below launches a local Neo4j database Docker container named ***testneo4j***. Let's break down what it does:

**Port Mapping:**
- 7474: Access the Neo4j web interface through your browser (usually http://localhost:7474).
- 7687: Enables communication with Neo4j using the Bolt protocol (essential for working with the database).

**Data Volumes:** Folders on your machine ($HOME/neo4j/*) store your database, logs, imports, etc., so your data is safe even if the container stops.

**Secure Credentials:** Variables `{NEO4J_USER}` and `{NEO4J_PASSWORD}` set your Neo4j username and password for secure access.

In [ ]:
%env NEO4J_USER={NEO4J_USER}
%env NEO4J_PASSWORD={NEO4J_PASSWORD}

In [ ]:
! docker run --name testneo4j -p7474:7474 -p7687:7687 -d \
    -v $HOME/neo4j/data:/data \
    -v $HOME/neo4j/logs:/logs \
    -v $HOME/neo4j/import:/var/lib/neo4j/import \
    -v $HOME/neo4j/plugins:/plugins \
    --env NEO4J_AUTH=$NEO4J_USER/$NEO4J_PASSWORD \
    --env='NEO4JLABS_PLUGINS=["apoc"]' \
    neo4j:latest

In [8]:
# Check if Docker Container is running
! docker ps -a

CONTAINER ID   IMAGE          COMMAND                  CREATED      STATUS         PORTS                                                                                            NAMES
a78c9e1e40b3   neo4j:latest   "tini -g -- /startup…"   6 days ago   Up 4 seconds   0.0.0.0:7474->7474/tcp, :::7474->7474/tcp, 7473/tcp, 0.0.0.0:7687->7687/tcp, :::7687->7687/tcp   testneo4j


In [7]:
# Start the Container if it is not running
! docker start testneo4j

testneo4j


### Connect to Neo4J Database

This code block creates a Graph object instance, establishing a connection to the Neo4j database.
See **utils/NEO4J_Graph.py** for more information

In [9]:
graph = Graph(NEO4J_URL, NEO4J_USER, NEO4J_PASSWORD)

### Neo4J Data Loading Functions

In [10]:
# Create FHIR Resource nodes in Neo4j
def create_resource_node(resource:dict) -> str:
    # print(resource_to_node(resource))
    node_creation_cypher = resource_to_node(resource)
    query_result, runtime = graph.query(node_creation_cypher)
    
    if len(query_result[0]) != 1:
        print(query_result)
        raise Exception("Resource Node creation query result does not meet defined format.")
       
    node_id = query_result[0][0]
    return node_id

In [11]:
# Create Date Nodes and Edges between Resource Nodes and Date Nodes
def create_date_nodes_edges(resource):
    edges = []
    dates = set() # set is used here to make sure dates are unique

    # generated the cypher for creating the reference & date edges and capture dates
    node_edges, node_dates = resource_to_edges(resource)
    edges += node_edges
    dates.update(node_dates)
    
    # Create Date Nodes - 'MERGE' Skip if Node exists
    for date in dates:
        cypher = 'MERGE (n:Date {name:"' + date + '", id: "' + date + '"})'
        graph.query(cypher)
    
    # Connect Resource Nodes and Date Nodes via Edges 
    for edge in edges:
        try:
            graph.query(edge)
        except:
            print(f'Failed to create edge: {edge}')
    
    return len(dates), len(edges)

### Ingest Data to Neo4J

In [12]:
def load_fhir_neo4j(fhir_files_list):
    file_counter = 0
    
    for fhir_file in fhir_files_list:
        
        fhir_file_name = os.path.basename(fhir_file)
        file_counter += 1
        
        if file_counter > 1:
            print("\n")
        
        print(f'File {file_counter} of {num_of_files}: {fhir_file_name}')
        
        with open(fhir_file) as raw:
            bundle = json.load(raw)
            resources_entry_list = bundle['entry']
            
            num_of_resources = len(resources_entry_list)
            print(f'    - Number of resources = {num_of_resources}')
                
            resource_counter = 0
            for entry in resources_entry_list:
                resource_counter += 1
                
                resource = entry['resource']
                # print(resource)
                resource_id = resource["id"]
                resource_type = resource["resourceType"]
                
                # Skip Provenance Resource
                if resource_type == 'Provenance':
                    continue

                    
                print(f'    - Processing Resource {resource_counter} of {num_of_resources}: Resource-Type = {resource_type}, Resource-ID = {resource_id}', end="\r", flush=True)
                
                #### LOAD DATA INTO NEO4J ####
                
                # Create Resource Nodes
                node_id = create_resource_node(resource)
                # print (node_id)
                
                # Create Date Nodes and Edges to connect Resource Nodes to Date Nodes
                date_nodes_count, edges_count = create_date_nodes_edges(resource)

**Trigger Neo4J Data Ingestion**
<div class="alert alert-block alert-warning">
<b>⚠️ Important: Below step will take few minutes to complete! ⚠️</b>
</div>

In [13]:
ingest_data = False

if ingest_data:
    start_time = time.time()
    load_fhir_neo4j(fhir_files_list)
    end_time = time.time()

    total_duration = str(datetime.timedelta(seconds = end_time - start_time))
    print(f'\nData Loading Completed in {total_duration}')
else:
    print(f'Set ingest_data to True to ingest data. Currently ingest_data={ingest_data}')

Set ingest_data to True to ingest data. Currently ingest_data=False


### Neo4j Database Helper Cells

The following three cells contain database management functions. For a new or blank database, you can skip them.

In [15]:
# Get type and number of each FHIR resource in the database
resource_metrics = graph.resource_metrics()
resource_metrics.sort()
pprint(resource_metrics)

[['AllergyIntolerance', 23],
 ['CarePlan', 66],
 ['CareTeam', 66],
 ['Claim', 1245],
 ['Condition', 526],
 ['Device', 23],
 ['DiagnosticReport', 1451],
 ['DocumentReference', 743],
 ['Encounter', 743],
 ['ExplanationOfBenefit', 1245],
 ['ImagingStudy', 13],
 ['Immunization', 299],
 ['Medication', 63],
 ['MedicationAdministration', 63],
 ['MedicationRequest', 502],
 ['Observation', 5252],
 ['Patient', 20],
 ['Procedure', 1513],
 ['SupplyDelivery', 184]]


In [16]:
# metrics for counting nodes and relationships
node_count, relationship_count = graph.database_metrics()
print('Database Metrics:')
print(f'    - Node Count = {node_count}')
print(f'    - Relationship Count = {relationship_count}')

Database Metrics:
    - Node Count = 15497
    - Relationship Count = 78165


- Wipe Database - Deletes all nodes and their relationships in database

<div class="alert alert-block alert-warning">
<b>⚠️ Warning: This command completely erases the Neo4j database. Proceed with extreme caution. ⚠️</b>
</div>

### Fetch Resource Nodes from Neo4J

In [17]:
def get_nodes_in_batches(graph, batch_size=1000):
    skip = 0
    while True:
        query_string = f'MATCH (r:resource) RETURN r SKIP {skip} LIMIT {batch_size}'        
        # print(query_string)
        results = graph.query(query_string)
        nodes = results[0]
        
        if not nodes:
            break  # No more nodes to fetch
        
        yield nodes  # Process this batch
        skip += batch_size

In [18]:
# print report on Resource nodes
batch_counter = 0
resource_counter = 0
resources_types_dict = {}

for batch in get_nodes_in_batches(graph):
    batch_counter += 1
    # print(f'Batch: {batch_counter}, Len of Batch = {len(batch)}')
    
    for node in batch:
        resource_counter += 1
        resource_type = node[0]['resource_type']
        
        if resource_type not in resources_types_dict.keys():
            resources_types_dict[resource_type] = 0
        
        resources_types_dict[resource_type] += 1
        # print(node[0])

sorted_dict = dict(sorted(resources_types_dict.items()))
pprint(sorted_dict)
print(f'Total Resource Nodes = {resource_counter}')

{'AllergyIntolerance': 23,
 'CarePlan': 66,
 'CareTeam': 66,
 'Claim': 1245,
 'Condition': 526,
 'Device': 23,
 'DiagnosticReport': 1451,
 'DocumentReference': 743,
 'Encounter': 743,
 'ExplanationOfBenefit': 1245,
 'ImagingStudy': 13,
 'Immunization': 299,
 'Medication': 63,
 'MedicationAdministration': 63,
 'MedicationRequest': 502,
 'Observation': 5252,
 'Patient': 20,
 'Procedure': 1513,
 'SupplyDelivery': 184}
Total Resource Nodes = 14040


---
## Load FHIR Data into Vector Search
[Vertex AI Vector Search Docs](https://cloud.google.com/vertex-ai/docs/vector-search/overview)

### Create GCS Bucket 

The Google Cloud Storage Bucket will be used by Vector Store Index.
[Create and manage your index](https://cloud.google.com/vertex-ai/docs/vector-search/create-manage-index)

In [ ]:
! set -x && gsutil mb -p $PROJECT_ID -l $REGION gs://$ME_EMBEDDING_GCS_DIR

### Import Google LLM Models

Load the pre-trained models textembedding-gecko@003 (embedding generation)
Learn more about Google's Foundation Models and their capabilities in this [documentation](https://cloud.google.com/vertex-ai/generative-ai/docs/learn/models#foundation_model_apis).

In [19]:
# Create Text Embedding
embeddings = VertexAIEmbeddings(
    model_name="textembedding-gecko@003",
    project=PROJECT_ID,
    location=REGION,
    max_retries=6
)

embeddings

INFO:google.auth.compute_engine._metadata:Compute Engine Metadata server call to universe/universe_domain returned 404, reason: <!DOCTYPE html>
<html lang=en>
  <meta charset=utf-8>
  <meta name=viewport content="initial-scale=1, minimum-scale=1, width=device-width">
  <title>Error 404 (Not Found)!!1</title>
  <style>
    *{margin:0;padding:0}html,code{font:15px/22px arial,sans-serif}html{background:#fff;color:#222;padding:15px}body{margin:7% auto 0;max-width:390px;min-height:180px;padding:30px 0 15px}* > body{background:url(//www.google.com/images/errors/robot.png) 100% 5px no-repeat;padding-right:205px}p{margin:11px 0 22px;overflow:hidden}ins{color:#777;text-decoration:none}a img{border:0}@media screen and (max-width:772px){body{background:none;margin-top:0;max-width:none;padding-right:0}}#logo{background:url(//www.google.com/images/branding/googlelogo/1x/googlelogo_color_150x54dp.png) no-repeat;margin-left:-5px}@media only screen and (min-resolution:192dpi){#logo{background:url(//ww

VertexAIEmbeddings(client=<vertexai.language_models.TextEmbeddingModel object at 0x7f74d86178b0>, project='propane-crawler-363311', location='us-central1', request_parallelism=5, max_retries=6, stop=None, model_name='textembedding-gecko@003', client_preview=None, temperature=None, max_output_tokens=None, top_p=None, top_k=None, credentials=None, n=1, streaming=False, safety_settings=None, instance={'max_batch_size': 250, 'batch_size': 250, 'min_batch_size': 5, 'min_good_batch_size': 5, 'lock': <unlocked _thread.lock object at 0x7f74cb3017c0>, 'batch_size_validated': False, 'task_executor': <concurrent.futures.thread.ThreadPoolExecutor object at 0x7f74d891f760>, 'embeddings_task_type_supported': True, 'get_embeddings_with_retry': <function TextEmbeddingModel.get_embeddings at 0x7f74c9f23e20>})

### Create Vertex AI Vector Search Index

In this step, we'll create the Vector Search Index.

For more details, refer to the Create and Manage Index: https://cloud.google.com/vertex-ai/docs/vector-search/create-manage-index documentation.
Vector Store supports two index types:

- **Batch Updates:** Ideal for less frequent modifications.
- **Streaming Updates:** Allows near-real-time additions and queries (our choice for this example).

- Let's create a dummy embeddings file required to initialize the Vector Search Index.

In [ ]:
# Create Temp folder
! mkdir $HOME/temp

# Create Dummy Embdeddng Data
dummy_embedding = {"id": str(uuid.uuid4()), "embedding": list(np.zeros(ME_DIMENSIONS))}

# Write Dummy Embedding Data to a JSON file
with open('./temp/dummy_embedding.json', 'w') as f:
    json.dump(dummy_embedding, f)
    
# Copy the dummy_embedding.json to Cloud Storage Bucket.
! set -x && gsutil cp ./temp/dummy_embedding.json gs://{ME_EMBEDDING_GCS_DIR}/init_index/dummy_embedding.json

- Let us now create a [Streaming Index](https://cloud.google.com/vertex-ai/docs/vector-search/create-manage-index#create-stream-index)

<div class="alert alert-block alert-warning">
<b>⚠️ Important: Index creation may take a few minutes. ⚠️</b>
</div>

In [20]:
me_utils = MatchingEngineUtils(PROJECT_ID, REGION, ME_INDEX_NAME)

me_index = me_utils.create_index(
    embedding_gcs_uri=f'gs://{ME_EMBEDDING_GCS_DIR}/init_index',
    dimensions=ME_DIMENSIONS,
    index_update_method='streaming',
    index_algorithm='tree-ah',
    shard_size= ME_SHARD_SIZE,
    distance_measure_type=ME_DISTANCE_MEASURE_TYPE,
    description=ME_DESCRIPTION
)
# me_index

INFO:google.auth.compute_engine._metadata:Compute Engine Metadata server call to universe/universe_domain returned 404, reason: <!DOCTYPE html>
<html lang=en>
  <meta charset=utf-8>
  <meta name=viewport content="initial-scale=1, minimum-scale=1, width=device-width">
  <title>Error 404 (Not Found)!!1</title>
  <style>
    *{margin:0;padding:0}html,code{font:15px/22px arial,sans-serif}html{background:#fff;color:#222;padding:15px}body{margin:7% auto 0;max-width:390px;min-height:180px;padding:30px 0 15px}* > body{background:url(//www.google.com/images/errors/robot.png) 100% 5px no-repeat;padding-right:205px}p{margin:11px 0 22px;overflow:hidden}ins{color:#777;text-decoration:none}a img{border:0}@media screen and (max-width:772px){body{background:none;margin-top:0;max-width:none;padding-right:0}}#logo{background:url(//www.google.com/images/branding/googlelogo/1x/googlelogo_color_150x54dp.png) no-repeat;margin-left:-5px}@media only screen and (min-resolution:192dpi){#logo{background:url(//ww

In [21]:
# Get information about the Index
if me_index:
    index_metadata = MessageToDict(me_index._pb)
    print('Index Details:')
    print(f'- Index Name = {index_metadata["name"]}')
    print(f'- Update Method = {index_metadata["indexUpdateMethod"]}')
    print(f'- Dimensions = {index_metadata["metadata"]["config"]["dimensions"]}')
    print(f'- Shard Size = {index_metadata["metadata"]["config"]["shardSize"]}')
    print(f'- Distance Measure Type = {index_metadata["metadata"]["config"]["distanceMeasureType"]}')
    algorithm = list(index_metadata["metadata"]["config"]['algorithmConfig'].keys())[0]
    print(f'- Algorithm = {algorithm}')
    # print(f'- Index Stats = {index_metadata["indexStats"]}')

# index_metadata

Index Details:
- Index Name = projects/884766917846/locations/us-central1/indexes/7150753036078415872
- Update Method = STREAM_UPDATE
- Dimensions = 768.0
- Shard Size = SHARD_SIZE_SMALL
- Distance Measure Type = DOT_PRODUCT_DISTANCE
- Algorithm = treeAhConfig


### Deploy Vector Search Index to an Index Endpoint

In this step, we'll deploy the Index to a Vector Search Index Endpoint. This endpoint is essential for sending queries to your index. 

We'll use a [public endpoint](https://cloud.google.com/vertex-ai/docs/vector-search/deploy-index-public) for this example. 

To set up a [Private Endpoint](https://cloud.google.com/vertex-ai/docs/vector-search/deploy-index-vpc), please refer to the documentation.

<div class="alert alert-block alert-warning">
<b>⚠️ Important: Deploying an Index to an Endpoint takes time, typically around 15 minutes or more. ⚠️</b>
</div>


In [22]:
me_endpoint = me_utils.deploy_index(
    machine_type=ME_ENDPOINT_MACHINE_TYPE,
    min_replica_count=ME_ENDPOINT_MIN_REPLICA_COUNT,
    max_replica_count=ME_ENDPOINT_MAX_REPLICA_COUNT,
    public_endpoint_enabled=True
)

INFO:google.auth.compute_engine._metadata:Compute Engine Metadata server call to universe/universe_domain returned 404, reason: <!DOCTYPE html>
<html lang=en>
  <meta charset=utf-8>
  <meta name=viewport content="initial-scale=1, minimum-scale=1, width=device-width">
  <title>Error 404 (Not Found)!!1</title>
  <style>
    *{margin:0;padding:0}html,code{font:15px/22px arial,sans-serif}html{background:#fff;color:#222;padding:15px}body{margin:7% auto 0;max-width:390px;min-height:180px;padding:30px 0 15px}* > body{background:url(//www.google.com/images/errors/robot.png) 100% 5px no-repeat;padding-right:205px}p{margin:11px 0 22px;overflow:hidden}ins{color:#777;text-decoration:none}a img{border:0}@media screen and (max-width:772px){body{background:none;margin-top:0;max-width:none;padding-right:0}}#logo{background:url(//www.google.com/images/branding/googlelogo/1x/googlelogo_color_150x54dp.png) no-repeat;margin-left:-5px}@media only screen and (min-resolution:192dpi){#logo{background:url(//ww

In [23]:
if me_endpoint:
    endpoint_metadata = MessageToDict(me_endpoint._pb)
    print(f'Endpoint Name: {endpoint_metadata["name"]}')
    print(f'Endpoint Public Domain Name: {endpoint_metadata["publicEndpointDomainName"]}')
    print('Deployed indexes on the endpoint:')
    
    for d in me_endpoint.deployed_indexes:
        print(f'    - Deployed Indexe ID = {d.id}')
        print(f'      Machine Type = {d.dedicated_resources.machine_spec.machine_type}')
        print(f'      Min Replica Count = {d.dedicated_resources.min_replica_count}')
        print(f'      Max Replica Count = {d.dedicated_resources.max_replica_count}')
        
# endpoint_metadata

Endpoint Name: projects/884766917846/locations/us-central1/indexEndpoints/2715411090560253952
Endpoint Public Domain Name: 1443686377.us-central1-884766917846.vdb.vertexai.goog
Deployed indexes on the endpoint:
    - Deployed Indexe ID = fhir_me_index_20240328062600
      Machine Type = e2-standard-2
      Min Replica Count = 2
      Max Replica Count = 10


### Ingest Data to VertexAI Vector Search

In [24]:
# Get Matching Engine Index id and Endpoint id
ME_INDEX_ID, ME_INDEX_ENDPOINT_ID = me_utils.get_index_and_endpoint()
print(f"ME_INDEX_ID={ME_INDEX_ID}")
print(f"ME_INDEX_ENDPOINT_ID={ME_INDEX_ENDPOINT_ID}")

ME_INDEX_ID=projects/884766917846/locations/us-central1/indexes/7150753036078415872
ME_INDEX_ENDPOINT_ID=projects/884766917846/locations/us-central1/indexEndpoints/2715411090560253952


In [25]:
#### Load Resource Text to Vector Search ####

def add_resource_text_embedding(me: MatchingEngine, resource_data):      
    
    resource_text_metadata = [
        {"namespace": "fhir_patient_id", "allow_list": [resource_data['patient_id']]},
        {"namespace": "fhir_resource_id", "allow_list": [resource_data['resource_id']]},
        {"namespace": "fhir_resource_type", "allow_list": [resource_data['resource_type']]},
        {"namespace": "neo4j_node_id", "allow_list": [resource_data['neo4j_id']]}
    ]

    # Add Resource Text Embeddings to Vector Search
    resource_text = resource_data['resource_text']
    if resource_text is not None:
        try:
            ids = me.add_texts(texts=[resource_text], metadatas=[resource_text_metadata])
            return ids
        except Exception as err:
            print(f"\nERROR: Unexpected while adding embeddings {err=}, {type(err)=}")
            print(f'Resource_Text = {resource_text}')
            print(f'Resource Metadata = {resource_text_metadata}')

In [26]:
def get_related_resource_node_texts(neo4j_node_id: str) -> str:
    contextualize_query = f"""
    MATCH (node :resource)
    WHERE elementId(node)='{neo4j_node_id}'
    MATCH(node)<-[]->(sc:resource)
    with node.text as self, reduce(s="", item in collect(distinct sc.text) | s + "\n\nSecondary Entry:\n" + item ) as ctxt limit 1
    return "Primary Entry:\n" + self + ctxt as text"""

    resource_text = graph.query(contextualize_query)[0][0][0]
    
    return resource_text

In [27]:
# Add Text Embeddings of all Resource Text to Vector Search

def create_embeddings_of_all_resource_text(me: MatchingEngine, enhanced_context=False):
    batch_counter = 0
    resource_counter = 0
    resources_types_dict = {}
    
    if enhanced_context:
        print('Enhanced Context Enabled')
    
    # Since we have large number of Resources, we iterate in batches
    for batch in get_nodes_in_batches(graph):
        batch_counter += 1
        # print(f'Batch: {batch_counter}, Len of Batch = {len(batch)}')
        
        for node in batch:
            resource_counter += 1
            resource_data = {
                'neo4j_id': node[0].element_id,
                'resource_id': node[0]['id'],
                'resource_type': node[0]['resource_type'],
                # 'resource_text': node[0]['text']        
            }
            
            # Get related Patient ID of the Resource
            resource_data['patient_id'] = ""
            if node[0]['resource_type'] == 'Patient':
                # print (node[0])
                resource_data['patient_id'] = node[0]['id']
            elif node[0]['subject_reference']:
                resource_data['patient_id'] = node[0]['subject_reference'].split(':')[-1]
            elif node[0]['patient_reference']: 
                resource_data['patient_id'] = node[0]['patient_reference'].split(':')[-1] 
            else:
                print(f"no patient id for resource_id: {node[0]['resource_id']}, resource_type: {node[0]['resource_type']}")
                # break
            
            if enhanced_context:
                resource_data['resource_text'] = get_related_resource_node_texts(node[0].element_id)
            else:
                resource_data['resource_text'] = node[0]['text']


            print(f"Processing patient_id: {resource_data['patient_id']} node_id: {resource_data['neo4j_id']}, resource_id={resource_data['resource_id']}, resource_type={resource_data['resource_type']}", end='\r', flush=True)

            # Create Embedding
            ids = add_resource_text_embedding(me, resource_data)
            print(f'Vector Search_ID: {ids}', end='\r', flush=True)
            # break
            
            resource_type = resource_data['resource_type']
            if resource_type not in resources_types_dict.keys():
                resources_types_dict[resource_type] = 0

            resources_types_dict[resource_type] += 1

        # break
    print(f'\nProcessed Resources: {resource_counter}')
    print('Resources Types Processed:')
    print(resources_types_dict)


In [28]:
# Initialize vector store
me = MatchingEngine.from_components(
    project_id=PROJECT_ID,
    region=REGION,
    gcs_bucket_name=f"gs://{ME_EMBEDDING_GCS_DIR}".split("/")[2],
    embedding=embeddings,
    index_id=ME_INDEX_ID,
    endpoint_id=ME_INDEX_ENDPOINT_ID,
)

INFO:google.auth.compute_engine._metadata:Compute Engine Metadata server call to universe/universe_domain returned 404, reason: <!DOCTYPE html>
<html lang=en>
  <meta charset=utf-8>
  <meta name=viewport content="initial-scale=1, minimum-scale=1, width=device-width">
  <title>Error 404 (Not Found)!!1</title>
  <style>
    *{margin:0;padding:0}html,code{font:15px/22px arial,sans-serif}html{background:#fff;color:#222;padding:15px}body{margin:7% auto 0;max-width:390px;min-height:180px;padding:30px 0 15px}* > body{background:url(//www.google.com/images/errors/robot.png) 100% 5px no-repeat;padding-right:205px}p{margin:11px 0 22px;overflow:hidden}ins{color:#777;text-decoration:none}a img{border:0}@media screen and (max-width:772px){body{background:none;margin-top:0;max-width:none;padding-right:0}}#logo{background:url(//www.google.com/images/branding/googlelogo/1x/googlelogo_color_150x54dp.png) no-repeat;margin-left:-5px}@media only screen and (min-resolution:192dpi){#logo{background:url(//ww

In the Cell below we are creating Embeddings of Resource Text and ingesting it into Vertex AI Vector Search.

<div class="alert alert-block alert-warning">
<b>⚠️ Important: Below step will take few minutes to complete! ⚠️</b>
</div>

In [29]:
ingest_data = False

if ingest_data:
    start_time = time.time()
    create_embeddings_of_all_resource_text(me, enhanced_context=False)
    end_time = time.time()

    total_duration = str(datetime.timedelta(seconds = end_time - start_time))
    print(f'\nData Loading Completed in {total_duration}')

else:
    print(f'Set ingest_data to True to ingest data. Currently ingest_data={ingest_data}')

Set ingest_data to True to ingest data. Currently ingest_data=False


### Query Vector Search Embeddings
In the step below we query the Vector Search DB to verify data load is successful.

In [30]:
me_utils = MatchingEngineUtils(PROJECT_ID, REGION, ME_INDEX_NAME)

# Get Matching Engine Index id and Endpoint id
ME_INDEX_ID, ME_INDEX_ENDPOINT_ID = me_utils.get_index_and_endpoint()

# Create Text Embedding
embeddings = VertexAIEmbeddings(
    model_name="textembedding-gecko@003",
    project=PROJECT_ID,
    location=REGION,
    max_retries=6
)

# Initialize vector store
me = MatchingEngine.from_components(
    project_id=PROJECT_ID,
    region=REGION,
    gcs_bucket_name=f"gs://{ME_EMBEDDING_GCS_DIR}".split("/")[2],
    embedding=embeddings,
    index_id=ME_INDEX_ID,
    endpoint_id=ME_INDEX_ENDPOINT_ID,
)

INFO:google.auth.compute_engine._metadata:Compute Engine Metadata server call to universe/universe_domain returned 404, reason: <!DOCTYPE html>
<html lang=en>
  <meta charset=utf-8>
  <meta name=viewport content="initial-scale=1, minimum-scale=1, width=device-width">
  <title>Error 404 (Not Found)!!1</title>
  <style>
    *{margin:0;padding:0}html,code{font:15px/22px arial,sans-serif}html{background:#fff;color:#222;padding:15px}body{margin:7% auto 0;max-width:390px;min-height:180px;padding:30px 0 15px}* > body{background:url(//www.google.com/images/errors/robot.png) 100% 5px no-repeat;padding-right:205px}p{margin:11px 0 22px;overflow:hidden}ins{color:#777;text-decoration:none}a img{border:0}@media screen and (max-width:772px){body{background:none;margin-top:0;max-width:none;padding-right:0}}#logo{background:url(//www.google.com/images/branding/googlelogo/1x/googlelogo_color_150x54dp.png) no-repeat;margin-left:-5px}@media only screen and (min-resolution:192dpi){#logo{background:url(//ww

In [31]:
# Test Vector Search Retrieval
text = "What can you tell me about Alina705's claim created on 2007-03-17?"
docs = me.similarity_search(text, k=5)

for doc in docs: 
    print(f'{doc.page_content}\nMetadata={doc.metadata}\n')
    

The type of information in this entry is observation. The status for this observation is final. The category of this observation is Survey. The code for this observation is Total score [AUDIT-C]. This observation was effective date time on 08/26/2015 at 18:20:41. This observation was issued on 08/26/2015 at 18:20:41. The value quantity for this observation is 0 {score}.
Metadata={'fhir_patient_id': '7dd2724f-7108-d053-5468-6663262b170b', 'fhir_resource_id': 'e778aad6-7761-0408-1f2b-58ef4113bd0d', 'fhir_resource_type': 'Observation', 'neo4j_node_id': '4:dfafac18-9f37-43c7-a973-b308f9812678:7452', 'score': 0.7911237478256226}

The type of information in this entry is observation. The status for this observation is final. The category of this observation is Survey. The code for this observation is Total score [AUDIT-C]. This observation was effective date time on 01/12/2017 at 18:55:46. This observation was issued on 01/12/2017 at 18:55:46. The value quantity for this observation is 0 {sc

---
## Preparing LLM Input: Prompts, Questions, Model
This section covers the design of LangChain Prompt Templates, User questions for LLM , and the choice of the LLM model for task execution

### Prompt Design

This cell defines the prompt template used to interact with the LLM. Try different prompts to see how they influence the LLM's output.

In [32]:
default_prompt='''
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
{context}
Human: {question}
'''

my_prompt='''
System: The following information contains entries about the patient. 
Use the primary entry and then the secondary entries to answer the user's question.
Each entry is its own type of data and secondary entries are supporting data for the primary one. 
You should restrict your answer to using the information in the entries provided. 

If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
{context}
----------------
User: {question}
'''

my_prompt_2='''
System: The context below contains entries about the patient's healthcare. 
Please limit your answer to the information provided in the context. Do not make up facts.
Please limit your answers only about the patient in the user question. If you do not find the patient name in the context.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
If you are asked about the patient's name and one the entries is of type patient, you should look for the first given name and family name and answer with: [given] [family]
----------------
{context}
Human: {question}
'''

prompt = PromptTemplate.from_template(my_prompt_2)

### Define User Questions

In [33]:
# question = "What can you tell me about Alfonso's claim created on 03/06/1977?"
# question = "What can you tell me about the medical claim created on 03/06/1977?"
# question = "Based on this explanation of benefits, how much did it cost and what service was provided?"
# question = "Based on this explanation of benefits created on July 15, 2016, how much did it cost and what service was provided?"
# question = "Based on this explanation of benefits created on March 6, 1978, how much did it cost and what service was provided?"
# question = "Based on this explanation of benefits created on January 11, 2009, how much did it cost and what service was provided?"
# question = "What was the blood pressure on 2/9/2014?"
# question = "What was the blood pressure?"
# question = "Based on this explanation of benefits created on January 18, 2014, how much did it cost and what service was provided?"
# question = "How much did the colon scan eighteen days after the first of the year 2019 cost?"
# question = "How much did the medical reconciliation on Dec. 29, 2023 cost?"
# question = "What can you tell me about Andrea7's claim created on 12/25/2003?"

# question = "What can you tell me about claim created on 12/25/2003?"
question = "What allergies does Antone63 have?"

---
## Testing Without RAG

The LLM would not be able to answer since it does not have the context. 
<br>Context is the Private User/Organization Data. FHIR Data in this example.

In [34]:
llm = VertexAI(model_name=VERTEX_AI_MODEL_NAME)

# Ask LLM the question
no_rag_answer = llm(question)
print(no_rag_answer)

Antone63 has not mentioned any allergies in their profile or any of their posts.


## Testing with RAG - Ask the LLM with Context

This cell will ask the LLM with the string representation of the resource node that is found by the vector index.

In [35]:
question

'What allergies does Antone63 have?'

In [36]:
# Create chain to answer questions
NUMBER_OF_RESULTS = 20
SEARCH_DISTANCE_THRESHOLD = 0.7

# Expose index to the retriever
retriever = me.as_retriever(
    search_type="similarity",
    search_kwargs={
        "k": NUMBER_OF_RESULTS,
        "search_distance": SEARCH_DISTANCE_THRESHOLD,
    },
)

In [37]:
vector_qa = RetrievalQA.from_chain_type(
    llm=ChatVertexAI(model_name=VERTEX_AI_MODEL_NAME),
    chain_type='stuff',
    retriever=retriever,
    # return_source_documents=True,
    verbose=True,
    chain_type_kwargs={"verbose": False, "prompt": prompt}
)

pprint(vector_qa.run(question))



> Entering new RetrievalQA chain...

> Finished chain.
('I cannot answer this question because the name Antone63 is not found in the '
 'context.')


### Cons of this approach
- The Vector Search similarity search fetches all matching Resource Types.
- However, the context does not include the Patients name for the retrieved Resources.
- Hence the LLM might summarize and extract the information from the context, but the Resources might not belong to a Patient in question.
- Sometimes the LLM responds "The context does not mention any patient".

## Testing with RAG - Ask the LLM with Enhanced Context

***Providing Context to the LLM***

We enrich the LLM's understanding by fetching text from linked resource nodes.

***Steps:***
- *Similarity Search:* Identify a matching resource's Neo4J Node ID in the Vector Search database.
- *Fetch Related Resources:* Query all nodes connected to the matching resource.
- *Concatenate Text:* Combine the text from all retrieved nodes to provide comprehensive context.

### Create a new Vector Search Index

In [38]:
# Create GCS Bucket for the new Enhanced Vector Search Index
! set -x && gsutil mb -p $PROJECT_ID -l $REGION gs://$ME_ENHANCED_EMBEDDING_GCS_DIR

+ gsutil mb -p propane-crawler-363311 -l us-central1 gs://propane-crawler-363311-me-bucket_enhanced
Creating gs://propane-crawler-363311-me-bucket_enhanced/...
ServiceException: 409 A Cloud Storage bucket named 'propane-crawler-363311-me-bucket_enhanced' already exists. Try another name. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.


In [39]:
# Create a new Vector Search Index for Enhanced Context
me_utils_enhanced = MatchingEngineUtils(PROJECT_ID, REGION, ME_ENHANCED_CONTEXT_INDEX_NAME)

me_index_enhanced = me_utils_enhanced.create_index(
    embedding_gcs_uri=f'gs://{ME_ENHANCED_EMBEDDING_GCS_DIR}/init_index',
    dimensions=ME_DIMENSIONS,
    index_update_method='streaming',
    index_algorithm='tree-ah',
    shard_size= ME_SHARD_SIZE,
    distance_measure_type=ME_DISTANCE_MEASURE_TYPE,
    description=ME_ENHANCED_DESCRIPTION
)

# me_index_enhanced

INFO:google.auth.compute_engine._metadata:Compute Engine Metadata server call to universe/universe_domain returned 404, reason: <!DOCTYPE html>
<html lang=en>
  <meta charset=utf-8>
  <meta name=viewport content="initial-scale=1, minimum-scale=1, width=device-width">
  <title>Error 404 (Not Found)!!1</title>
  <style>
    *{margin:0;padding:0}html,code{font:15px/22px arial,sans-serif}html{background:#fff;color:#222;padding:15px}body{margin:7% auto 0;max-width:390px;min-height:180px;padding:30px 0 15px}* > body{background:url(//www.google.com/images/errors/robot.png) 100% 5px no-repeat;padding-right:205px}p{margin:11px 0 22px;overflow:hidden}ins{color:#777;text-decoration:none}a img{border:0}@media screen and (max-width:772px){body{background:none;margin-top:0;max-width:none;padding-right:0}}#logo{background:url(//www.google.com/images/branding/googlelogo/1x/googlelogo_color_150x54dp.png) no-repeat;margin-left:-5px}@media only screen and (min-resolution:192dpi){#logo{background:url(//ww

In [40]:
# Get information about the Index
if me_index_enhanced:
    index_metadata = MessageToDict(me_index_enhanced._pb)
    print('Index Details:')
    print(f'- Index Name = {index_metadata["name"]}')
    print(f'- Update Method = {index_metadata["indexUpdateMethod"]}')
    print(f'- Dimensions = {index_metadata["metadata"]["config"]["dimensions"]}')
    print(f'- Shard Size = {index_metadata["metadata"]["config"]["shardSize"]}')
    print(f'- Distance Measure Type = {index_metadata["metadata"]["config"]["distanceMeasureType"]}')
    algorithm = list(index_metadata["metadata"]["config"]['algorithmConfig'].keys())[0]
    print(f'- Algorithm = {algorithm}')
    # print(f'- Index Stats = {index_metadata["indexStats"]}')

# index_metadata

Index Details:
- Index Name = projects/884766917846/locations/us-central1/indexes/8972459085349781504
- Update Method = STREAM_UPDATE
- Dimensions = 768.0
- Shard Size = SHARD_SIZE_SMALL
- Distance Measure Type = DOT_PRODUCT_DISTANCE
- Algorithm = treeAhConfig


In [41]:
me_endpoint_enhanced = me_utils_enhanced.deploy_index(
    machine_type=ME_ENDPOINT_MACHINE_TYPE,
    min_replica_count=ME_ENDPOINT_MIN_REPLICA_COUNT,
    max_replica_count=ME_ENDPOINT_MAX_REPLICA_COUNT,
    public_endpoint_enabled=True
)

INFO:google.auth.compute_engine._metadata:Compute Engine Metadata server call to universe/universe_domain returned 404, reason: <!DOCTYPE html>
<html lang=en>
  <meta charset=utf-8>
  <meta name=viewport content="initial-scale=1, minimum-scale=1, width=device-width">
  <title>Error 404 (Not Found)!!1</title>
  <style>
    *{margin:0;padding:0}html,code{font:15px/22px arial,sans-serif}html{background:#fff;color:#222;padding:15px}body{margin:7% auto 0;max-width:390px;min-height:180px;padding:30px 0 15px}* > body{background:url(//www.google.com/images/errors/robot.png) 100% 5px no-repeat;padding-right:205px}p{margin:11px 0 22px;overflow:hidden}ins{color:#777;text-decoration:none}a img{border:0}@media screen and (max-width:772px){body{background:none;margin-top:0;max-width:none;padding-right:0}}#logo{background:url(//www.google.com/images/branding/googlelogo/1x/googlelogo_color_150x54dp.png) no-repeat;margin-left:-5px}@media only screen and (min-resolution:192dpi){#logo{background:url(//ww

In [42]:
# Get Matching Engine Index id and Endpoint id
me_utils_enhanced = MatchingEngineUtils(PROJECT_ID, REGION, ME_ENHANCED_CONTEXT_INDEX_NAME)
ME_ENHANCED_INDEX_ID, ME_ENHANCED_INDEX_ENDPOINT_ID = me_utils_enhanced.get_index_and_endpoint()
print(f"ME_ENHANCED_INDEX_ID={ME_ENHANCED_INDEX_ID}")
print(f"ME_ENHANCED_INDEX_ENDPOINT_ID={ME_ENHANCED_INDEX_ENDPOINT_ID}")

INFO:google.auth.compute_engine._metadata:Compute Engine Metadata server call to universe/universe_domain returned 404, reason: <!DOCTYPE html>
<html lang=en>
  <meta charset=utf-8>
  <meta name=viewport content="initial-scale=1, minimum-scale=1, width=device-width">
  <title>Error 404 (Not Found)!!1</title>
  <style>
    *{margin:0;padding:0}html,code{font:15px/22px arial,sans-serif}html{background:#fff;color:#222;padding:15px}body{margin:7% auto 0;max-width:390px;min-height:180px;padding:30px 0 15px}* > body{background:url(//www.google.com/images/errors/robot.png) 100% 5px no-repeat;padding-right:205px}p{margin:11px 0 22px;overflow:hidden}ins{color:#777;text-decoration:none}a img{border:0}@media screen and (max-width:772px){body{background:none;margin-top:0;max-width:none;padding-right:0}}#logo{background:url(//www.google.com/images/branding/googlelogo/1x/googlelogo_color_150x54dp.png) no-repeat;margin-left:-5px}@media only screen and (min-resolution:192dpi){#logo{background:url(//ww

ME_ENHANCED_INDEX_ID=projects/884766917846/locations/us-central1/indexes/8972459085349781504
ME_ENHANCED_INDEX_ENDPOINT_ID=projects/884766917846/locations/us-central1/indexEndpoints/945918649468715008


In [43]:
# Initialize vector store
me_enhanced = MatchingEngine.from_components(
    project_id=PROJECT_ID,
    region=REGION,
    gcs_bucket_name=f"gs://{ME_ENHANCED_EMBEDDING_GCS_DIR}".split("/")[2],
    embedding=embeddings,
    index_id=ME_ENHANCED_INDEX_ID,
    endpoint_id=ME_ENHANCED_INDEX_ENDPOINT_ID,
)

INFO:google.auth.compute_engine._metadata:Compute Engine Metadata server call to universe/universe_domain returned 404, reason: <!DOCTYPE html>
<html lang=en>
  <meta charset=utf-8>
  <meta name=viewport content="initial-scale=1, minimum-scale=1, width=device-width">
  <title>Error 404 (Not Found)!!1</title>
  <style>
    *{margin:0;padding:0}html,code{font:15px/22px arial,sans-serif}html{background:#fff;color:#222;padding:15px}body{margin:7% auto 0;max-width:390px;min-height:180px;padding:30px 0 15px}* > body{background:url(//www.google.com/images/errors/robot.png) 100% 5px no-repeat;padding-right:205px}p{margin:11px 0 22px;overflow:hidden}ins{color:#777;text-decoration:none}a img{border:0}@media screen and (max-width:772px){body{background:none;margin-top:0;max-width:none;padding-right:0}}#logo{background:url(//www.google.com/images/branding/googlelogo/1x/googlelogo_color_150x54dp.png) no-repeat;margin-left:-5px}@media only screen and (min-resolution:192dpi){#logo{background:url(//ww

### Ingest Enhanced Context data to Vector Search

In the Cell below we are creating Embeddings of Resource Text and ingesting it into Vertex AI Vector Search.

<div class="alert alert-block alert-warning">
<b>⚠️ Important: Below step will take few minutes to complete! ⚠️</b>
</div>

In [44]:
# Trigger Enhanced Context Data Ingestion to vector Search
ingest_data = False

if ingest_data:
    create_embeddings_of_all_resource_text(me_enhanced, enhanced_context=True)
else:
    print(f'Set ingest_data to True to ingest data. Currently ingest_data={ingest_data}')

Set ingest_data to True to ingest data. Currently ingest_data=False


### RAG with Enhanced Context

In [101]:
# question = "What procedure was performed on Antone63 on 2014-04-20?"
question

'What allergies does Antone63 have?'

In [102]:
# Create chain to answer questions
NUMBER_OF_RESULTS = 10
SEARCH_DISTANCE_THRESHOLD = 0.6

# Expose index to the retriever
retriever_enhanced = me_enhanced.as_retriever(
    search_type="similarity",
    search_kwargs={
        "k": NUMBER_OF_RESULTS,
        "search_distance": SEARCH_DISTANCE_THRESHOLD,
    },
)

In [103]:
# question = "Based on this explanation of benefits created on February 11, 1999, how much did it cost and what service was provided?"
vector_qa_enhanced = RetrievalQA.from_chain_type(
    llm=ChatVertexAI(model_name=VERTEX_AI_MODEL_NAME),
    chain_type='stuff',
    retriever=retriever_enhanced,
    # return_source_documents=True,
    verbose=True,
    chain_type_kwargs={"verbose": True, "prompt": prompt}
)

# question = 'Tell me about the latest medical reconciliation?'
pprint(vector_qa_enhanced.run(question))



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

System: The context below contains entries about the patient's healthcare. 
Please limit your answer to the information provided in the context. Do not make up facts.
Please limit your answers only about the patient in the user question. If you do not find the patient name in the context.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
If you are asked about the patient's name and one the entries is of type patient, you should look for the first given name and family name and answer with: [given] [family]
----------------
Primary Entry:
The type of information in this entry is allergy intolerance. The clinical status for this allergy intolerance is active. The verification status for this allergy intolerance is confirmed. The type of this allergy intolerance is allergy. The category of this allergy i

### Cons of this approach
A problem with this approach is the data is repeated multiple times (e.g. Patient Information). This results in increased LLM Token consumption and costs.
An alternate strategy would be:
- First retrieve the Resource in Question from Vector Search
- Use the Resource ID to query Neo4J for related Resources (Text field)
- Remove Duplicate Resource Entries 
- Dynamically construct the Context and pass it to the LLM.

## Testing with RAG - Custom Retreiver

The Custom retriever addresses the above problem. It does so by:
- Identifying the Patient Name from User Query. If Patient name not present prompt the User for the Patient Name
- Idenitfy the FHIR Resource Type in Question
- Perform a similarity search by narrowing down the results based on Patient ID and Resource Type

Thus increasing the accuracy of the Context provided to the LLM for information extraction.

**Enhancement** - This approach does not address the below use case of getting Related Resources for more context. This can be fixed by combining the Custom Retriever and fetching related Resource text from Neo4J to provide the necessary context to the LLM.

In [114]:
me_utils = MatchingEngineUtils(PROJECT_ID, REGION, ME_INDEX_NAME)

# Get Matching Engine Index id and Endpoint id
ME_INDEX_ID, ME_INDEX_ENDPOINT_ID = me_utils.get_index_and_endpoint()

# Create Text Embedding
embeddings = VertexAIEmbeddings(
    model_name="textembedding-gecko@003",
    project=PROJECT_ID,
    location=REGION,
    max_retries=6
)

# Initialize vector store
me = MatchingEngine.from_components(
    project_id=PROJECT_ID,
    region=REGION,
    gcs_bucket_name=f"gs://{ME_EMBEDDING_GCS_DIR}".split("/")[2],
    embedding=embeddings,
    index_id=ME_INDEX_ID,
    endpoint_id=ME_INDEX_ENDPOINT_ID,
)

INFO:google.auth.compute_engine._metadata:Compute Engine Metadata server call to universe/universe_domain returned 404, reason: <!DOCTYPE html>
<html lang=en>
  <meta charset=utf-8>
  <meta name=viewport content="initial-scale=1, minimum-scale=1, width=device-width">
  <title>Error 404 (Not Found)!!1</title>
  <style>
    *{margin:0;padding:0}html,code{font:15px/22px arial,sans-serif}html{background:#fff;color:#222;padding:15px}body{margin:7% auto 0;max-width:390px;min-height:180px;padding:30px 0 15px}* > body{background:url(//www.google.com/images/errors/robot.png) 100% 5px no-repeat;padding-right:205px}p{margin:11px 0 22px;overflow:hidden}ins{color:#777;text-decoration:none}a img{border:0}@media screen and (max-width:772px){body{background:none;margin-top:0;max-width:none;padding-right:0}}#logo{background:url(//www.google.com/images/branding/googlelogo/1x/googlelogo_color_150x54dp.png) no-repeat;margin-left:-5px}@media only screen and (min-resolution:192dpi){#logo{background:url(//ww

In [115]:
question

'What allergies does Antone63 have?'

In [46]:


llm = VertexAI(model_name=VERTEX_AI_MODEL_NAME)

graph = Graph(NEO4J_URL, NEO4J_USER, NEO4J_PASSWORD)
retriever = FHIRResourcesRetriever(llm=llm, me=me, neo4j_graph=graph)

vector_qa = RetrievalQA.from_chain_type(
    llm=ChatVertexAI(model_name=VERTEX_AI_MODEL_NAME),
    chain_type='stuff',
    retriever=retriever,
    # return_source_documents=True,
    verbose=True,
    chain_type_kwargs={"verbose": False, "prompt": prompt}
)

# print(vector_qa.__dir__())
print(vector_qa.invoke(question))



> Entering new RetrievalQA chain...

> Finished chain.
{'query': 'What allergies does Antone63 have?', 'result': 'Antone63 has allergies to the following substances:\n- Allergy to substance (finding)\n- Grass pollen (substance)\n- Mold (organism)\n- Aspirin\n- Animal dander (substance)\n- Soy bean'}


## RAG with Enhanced Context and Custom Retriever

In [107]:
question

'What allergies does Antone63 have?'

In [108]:
llm = VertexAI(model_name=VERTEX_AI_MODEL_NAME)

graph = Graph(NEO4J_URL, NEO4J_USER, NEO4J_PASSWORD)
retriever = FHIRResourcesRetriever(llm=llm, me=me_enhanced, neo4j_graph=graph)

vector_qa = RetrievalQA.from_chain_type(
    llm=ChatVertexAI(model_name=VERTEX_AI_MODEL_NAME),
    chain_type='stuff',
    retriever=retriever,
    # return_source_documents=True,
    verbose=True,
    chain_type_kwargs={"verbose": True, "prompt": prompt}
)

print(vector_qa.invoke(question))



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

System: The context below contains entries about the patient's healthcare. 
Please limit your answer to the information provided in the context. Do not make up facts.
Please limit your answers only about the patient in the user question. If you do not find the patient name in the context.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
If you are asked about the patient's name and one the entries is of type patient, you should look for the first given name and family name and answer with: [given] [family]
----------------
The Patient name is Antone63 
FHIR Resource Type is AllergyIntolerance 
Patient_ID=ff8b984d-7727-92f0-0995-ae25d0eb774f 
Below is the medical information of Antone63: 
[Document(page_content='Primary Entry:\nThe type of information in this entry is allergy intolerance. The clinical 

-----

In [ ]:
def date_for_question(question_to_find_date):
    _llm = text_bison_llm 
    _response = _llm(f'''
    system:Given the following question from the user, extract the date the question is asking about.
    Return the answer formatted as JSON only, as a single line.
    Use the form:
    
    {{"date":"[THE DATE IN THE QUESTION]"}}
    
    Use the date format of month/day/year.
    Use two digits for the month and day.
    Use four digits for the year.
    So 3/4/23 should be returned as {{"date":"03/04/2023"}}.
    So 04/14/89 should be returned as {{"date":"04/14/1989"}}.
    
    Please do not include any special formatting characters, like new lines or "\\n".
    Please do not include the word "json".
    Please do not include triple quotes.
    
    If there is no date, do not make one up. 
    If there is no date return the word "none", like: {{"date":"none"}}
    
    user:{question_to_find_date}
    ''')
    date_json = json.loads(_response)
    return date_json['date']

date_str = date_for_question(question)
print(date_str)

In [65]:
def get_patient_name(question_to_find_names):
    # _llm = text_bison_llm 
    
    _response = llm(f'''
    system:Given the following question from the user, identify all potential first and last names within this sentence.
    
    The name might also contain numbers e.g. Andrea7, Jenkins714, Chasity985, Pagac496
    The name might also contain Apostrophe e.g. Andrea's, John's, Johns' James'
    If the nameis in the format Smith, John then first-name = John, last-name = Smith
    
    Return the answer formatted as first-name last-name.
    
    Use the form:
    first-name last name
    
    Please do not include any special formatting characters, like new lines or "\\n".
    Please do not include triple quotes.
    
    If there are no names, do not make one up. 
    If there are no names return an empty string link ""
    
    user:{question_to_find_names}
    ''')
    names = _response
    if names == "":
        input("Please enter the Patient name:")
    return names

# Jenkins714 Andrea7
q = question
patient_name = get_patient_name(q).strip()
print(patient_name)


# While Loading data into Neo4J we created a 'Text' Attribute to convert Resource JSON format to Text.
# We will use the same sentence structure to get an accurate seearch result from Vector Store.
patient_name_query=f"The type of information in this entry is patient. The name use for this patient is official. The name family for this patient is {patient_name}. The name given 0 for this patient is {patient_name}."
# print(patient_name_query)

response = me.similarity_search(patient_name_query, k=2)
for doc in response:
    if doc.metadata['score'] > .85:
        print(doc)
        print("\n")

Antone63


---
## Cleaning Up

<div class="alert alert-block alert-warning">
<b>⚠️ Important: To avoid incurring charges, please delete the Google Cloud resources used in this tutorial. ⚠️</b>
</div>



In [ ]:
CLEANUP_RESOURCES = True

### Delete Neo4J Docker

In [ ]:
# Wipe Neo4J Database
graph = Graph(NEO4J_URL, NEO4J_USER, NEO4J_PASSWORD)
if CLEANUP_RESOURCES:
    graph.wipe_database()

In [ ]:
# DELETE NEO4J CONTAINER
if CLEANUP_RESOURCES:
    ! docker stop testneo4j
    ! docker rm -fv testneo4j
    ! sudo rm -rf $HOME/neo4j

### Delete Vector Search Indexes & Index-Endpoints

- Delete ME Vector Search Index and Endpoints

In [ ]:
me_utils = MatchingEngineUtils(PROJECT_ID, REGION, ME_INDEX_NAME)
ME_INDEX_ID, ME_INDEX_ENDPOINT_ID = me_utils.get_index_and_endpoint()

# Delete Endpoint
if CLEANUP_RESOURCES and "me_utils" in globals():
    print(
        f"Undeploying all deployed indexes and deleting the index endpoint {ME_INDEX_ENDPOINT_ID}"
    )
    me_utils.delete_index_endpoint()

# Delete Index     
if CLEANUP_RESOURCES and "me_utils" in globals():
    print(f"Deleting the index {ME_INDEX_ID}")
    me_utils.delete_index()    

# Delete Bucket    
if CLEANUP_RESOURCES:
    # Delete contents of the bucket 
    ! gsutil -m rm -r gs://{ME_EMBEDDING_GCS_DIR}
    ! gsutil rb gs://{ME_EMBEDDING_GCS_DIR}

print('Vector Search and GCS Bucket Cleaning complete!')

- Delete ME_ENHANCED Vector Search Index and Endpoints

In [ ]:
me_utils_enhanced = MatchingEngineUtils(PROJECT_ID, REGION, ME_ENHANCED_CONTEXT_INDEX_NAME)
ME_ENHANCED_INDEX_ID, ME_ENHANCED_INDEX_ENDPOINT_ID = me_utils_enhanced.get_index_and_endpoint()

# Delete Endpoint
if CLEANUP_RESOURCES and "me_utils_enhanced" in globals():
    print(
        f"Undeploying all deployed indexes and deleting the index endpoint {ME_ENHANCED_INDEX_ENDPOINT_ID}"
    )
    me_utils_enhanced.delete_index_endpoint()

# Delete Index    
if CLEANUP_RESOURCES and "me_utils_enhanced" in globals():
    print(f"Deleting the index {ME_ENHANCED_INDEX_ID}")
    me_utils_enhanced.delete_index()

# Delete Bucket
if CLEANUP_RESOURCES:
    ! gsutil -m rm -r gs://{ME_ENHANCED_EMBEDDING_GCS_DIR}
    ! gsutil rb gs://{ME_ENHANCED_EMBEDDING_GCS_DIR}

### Delete the Google Cloud Storage Bucket

In [ ]:
if CLEANUP_RESOURCES:

    # Delete Bucket
    ! gsutil rb gs://{ME_EMBEDDING_GCS_DIR}
    ! gsutil rb gs://{ME_ENHANCED_EMBEDDING_GCS_DIR}

# To-Do

- Example MedLM Integration - Query all Patients with high risk of some disease (Stroke)
- Query all Vacation candiates and based on time send Notification to them. E.g. Child vacccination message to Parent - Query Neo4J Date Nodes range